# 数据预处理

## 读取数据
手工创建一个csv数据集, 并进行读取

In [2]:
import os

os.makedirs(os.path.join('..', 'data/Chap2/2.2'), exist_ok=True)
data_file = os.path.join('..', 'data/Chap2/2.2', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 首行指定列名
    f.write('NA,Pave,127500\n')  # 每行代表一个样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')
print("Successfully created file: " + data_file)

Successfully created file: ../data/Chap2/2.2/house_tiny.csv


通过上段代码创建了一个房价数据集, 每一列分别代表房间数量/ 街道类型/ 房屋价格

下面使用pandas对其进行读取

In [3]:
import pandas as pd
data = pd.read_csv(data_file)
print("Data: \n", data)

# 注意，上面创建data_file时用NA表示缺失值, 且源文件中显示的确实是NA,
# 而此处用NaN表示缺失值, 与源文件不同, 可能是pandas进行了转换

Data: 
    NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 处理缺失值

常见的处理缺失值方法包括:
1. 插值法: 用一个值代替缺失值
2. 忽略法: 直接忽略缺失值

此处使用插值法

In [12]:
inputs = data.iloc[:, :2]   # 前2列
outputs = data.iloc[:, 2]   # 第3列
print("Inputs: \n", inputs, end='\n\n')
print("Outputs: \n", outputs, end='\n\n')

Inputs: 
    NumRooms Alley
0       NaN  Pave
1       2.0   NaN
2       4.0   NaN
3       NaN   NaN

Outputs: 
 0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64



In [13]:
# 用均值来填充缺失值
inputs = inputs.fillna(inputs.mean())
print("Fill with mean: \n", inputs, end='\n\n')

Fill with mean: 
    NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN



/tmp/ipykernel_3982996/369478748.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  inputs = inputs.fillna(inputs.mean())


可以发现第二列Alley不是数字, 所以考虑除了均值填充之外的方法

由于Alley列只接受两种类型的值: Pave 和 NaN, pandas可以自动将此列转换为两列 Alley_Pave 和 Alley_nan

那么对于Alley列的值为Pave的行，Alley_Pave=1; 反之同理

In [14]:
# get_dummies将非数值的列转换为数值
# 原理(One-Hot Encoding): 
# 为每个类创建一个新的列, 用0或1表示是否属于该类
inputs = pd.get_dummies(inputs, dummy_na=True)
print("One-Hot Encoding: \n", inputs, end='\n\n')

One-Hot Encoding: 
    NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1



## 转换为张量

In [15]:
# 通过to_numpy()将pandas的DataFrame转换为numpy的ndarray
# 再用torch.tensor()将numpy的ndarray转换为pytorch的tensor
import torch
X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
print("X: \n", X, end='\n\n')
print("y: \n", y, end='\n\n')

X: 
 tensor([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]], dtype=torch.float64)

y: 
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64)

